In [1]:
import os
import sys
home_dir = "../../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd

from data_loader import get_merged_scores_unidirectional_df
from utils.data_dicts import all_method_names
from utils.performance_metrics import *

In [2]:
# Plans: 
#     For Pathogenic + Neutral/Baseline variants,

#     1. Take all proteins containing pathogenic variants.
#     2. Take all baseline variants for this protein set.
#     3. If a protein does not have baseline variants, discard it. You will end up with X proteins.
#     4. This is the new dataset – count and list the number of proteins and pathogenic, likely-pathogenic, common, and rare variants.
#     5. To calculate performance,
#         1. sample one pathogenic variant and one neutral variant for each of the X proteins
#         2. Calculate all the performance metrics.
#         3. Repeat the above two steps ten times and compute mean & std.

#     For Likely-pathogenic + Neutral variants, please repeat above steps with likely-pathogenic variants.
#     Finally, combine two datasets (A, B), count and list the number of proteins and pathogenic, likely-pathogenic, common, and rare variants.
#     Please send me the updated Performance metrics sheet and consolidated prediction sheet (with prediction scores) for the new combined (A, B) variants.

In [6]:
task = "patho"

In [12]:
xresult_df = get_merged_scores_unidirectional_df(task, home_dir) # not using this

Index(['clinvar_id', 'gene_name', 'gene_id', 'snp_id', 'mrna_acc_version',
       'mrna_gi', 'prot_variant', 'prot_acc_version', '1indexed_prot_mt_pos',
       'wt_aa', 'mt_aa', 'wt_aa_1letter', 'mt_aa_1letter', 'chrom_variant',
       'chrom_acc_version', 'chrom_num', 'chrom_pos', 'ref_allele',
       'alt_allele', 'class', 'metarnn', 'mvp', 'sift', 'polyphen2_HVAR',
       'cadd_raw', 'revel', 'integrated_fitCons', 'phyloP17way_primate',
       'phastCons17way_primate', 'bStatistic', 'esm1b_t33_650M_UR50S',
       'esm1v_t33_650M_UR90S', 'esm2_t33_650M_UR50D', 'prottrans_bert_bfd',
       'prottrans_albert_bfd', 'plus_rnn', 'prottrans_t5_xl_u50', 'mut_real',
       'vespa', 'vespal', 'proteinbert', 'sequnet', 'protbert', 'unirep',
       'conservation'],
      dtype='object')
(12263, 45)
Likely-pathogenic    4804
Rare                 3073
Pathogenic           2499
Common               1887
Name: class, dtype: int64


In [2]:
result_df = pd.read_csv(home_dir+f"data/merged_predictions_with_alphamissense/dfPatho_addedAM.tsv", sep="\t")
print(result_df.columns)
print(result_df.shape)
print(result_df["class"].value_counts())

Index(['clinvar_id', 'gene_name', 'gene_id', 'snp_id', 'mrna_acc_version',
       'mrna_gi', 'prot_variant', 'prot_acc_version', '1indexed_prot_mt_pos',
       'wt_aa', 'mt_aa', 'wt_aa_1letter', 'mt_aa_1letter', 'chrom_variant',
       'chrom_acc_version', 'chrom_num', 'chrom_pos', 'ref_allele',
       'alt_allele', 'class', 'metarnn', 'mvp', 'sift', 'polyphen2_HVAR',
       'cadd_raw', 'revel', 'integrated_fitCons', 'phyloP17way_primate',
       'phastCons17way_primate', 'bStatistic', 'esm1b_t33_650M_UR50S',
       'esm1v_t33_650M_UR90S', 'esm2_t33_650M_UR50D', 'prottrans_bert_bfd',
       'prottrans_albert_bfd', 'plus_rnn', 'prottrans_t5_xl_u50', 'mut_real',
       'vespa', 'vespal', 'proteinbert', 'sequnet', 'protbert', 'unirep',
       'conservation', 'SIFT', 'PolyPhen2', 'MetaRNN', 'Revel', 'MVP', 'CADD',
       'Conservation\n(PhastCons)', 'ESM1b', 'ESM1v', 'ESM2', 'Prottrans BERT',
       'Prottrans T5', 'VESPA', 'ProteinBERT', 'Seq-Unet', 'ProtBERT',
       'UniRep', 'PLUS-RNN'

/tmp/ipykernel_1776587/1568287625.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  result_df = pd.read_csv(home_dir+f"data/merged_predictions_with_alphamissense/dfPatho_addedAM.tsv", sep="\t")


In [4]:
all_method_names = all_method_names + ["am_pathogenicity_uniqId_Cpos", "am_pathogenicity_uniqId_CposAA", "am_pathogenicity_uniqId_Tpos"] + ["random_classifier"]
all_method_names

['sift',
 'polyphen2_HVAR',
 'metarnn',
 'revel',
 'mvp',
 'cadd_raw',
 'integrated_fitCons',
 'phyloP17way_primate',
 'phastCons17way_primate',
 'bStatistic',
 'esm1b_t33_650M_UR50S',
 'esm1v_t33_650M_UR90S',
 'esm2_t33_650M_UR50D',
 'prottrans_bert_bfd',
 'prottrans_albert_bfd',
 'plus_rnn',
 'prottrans_t5_xl_u50',
 'vespa',
 'vespal',
 'proteinbert',
 'sequnet',
 'protbert',
 'unirep',
 'conservation',
 'am_pathogenicity_uniqId_Cpos',
 'am_pathogenicity_uniqId_CposAA',
 'am_pathogenicity_uniqId_Tpos',
 'random_classifier']

In [11]:
# Pathogenic, Likely-pathogenic
positive_cls, negative_cls, n_runs, n_samples, fill_missing_with_median = "Pathogenic", "Neutral", 10, None, False

pos_cls_variants_df = result_df[result_df["class"]==positive_cls]
neg_cls_variants_df = result_df[((result_df["class"]=="Common") | (result_df["class"]=="Rare"))]
print(f"pos variants: {pos_cls_variants_df.shape}, neg variants: {neg_cls_variants_df.shape}")

pos_cls_prots = result_df[result_df["class"]==positive_cls]["prot_acc_version"].unique()
neg_cls_prots = result_df[((result_df["class"]=="Common") | (result_df["class"]=="Rare"))]["prot_acc_version"].unique()

pos_cls_prots, neg_cls_prots = set(pos_cls_prots), set(neg_cls_prots)
prots_set = pos_cls_prots.intersection(neg_cls_prots)
# print(len(pos_cls_prots), len(neg_cls_prots), len(prots_set))
print(f"#-proteins\t{len(prots_set)}")

pos_cls_variants_df = result_df[result_df["prot_acc_version"].isin(prots_set) & (result_df["class"]==positive_cls)]
neg_cls_variants_df = result_df[result_df["prot_acc_version"].isin(prots_set) & ((result_df["class"]=="Common") | (result_df["class"]=="Rare"))]
print(pos_cls_variants_df.shape, neg_cls_variants_df.shape)

variants_df = pd.concat([pos_cls_variants_df, neg_cls_variants_df])
print(variants_df["prot_acc_version"].unique().shape)


variants_df["class"].value_counts()

pos variants: (2499, 79), neg variants: (4960, 79)
#-proteins	754
(1991, 79) (3173, 79)
(754,)


Pathogenic    1991
Rare          1900
Common        1273
Name: class, dtype: int64

In [12]:
variants_df.to_csv(home_dir+f"data/performance_analysis_patho/data_{positive_cls}_vs_{negative_cls}.tsv", sep="\t", index=False, header=True)

In [9]:
def get_pathogenic_cls_analysis_threshold(method_name, home_dir=""):
    patho_performance_metrics_df = pd.read_csv(home_dir + f"data/performance_analysis_patho/patho_Pathogenic_vs_Neutral.tsv", sep="\t")  
    # performance_analysis, performance_analysis_minority_cls, performance_analysis_alphamissense, performance_analysis_alphamissense_minority_cls
    patho_th_max = patho_performance_metrics_df[patho_performance_metrics_df["Models\\Metrics"] == method_name]["Th-max"].values[1]
    patho_th_max = patho_th_max.split("(")[0]
    patho_th_max = float(patho_th_max)
    # print(f"\tComputed th from pathogenic-analysis: {patho_th_max}")
    return patho_th_max

def fill_missing_values(df, method_name, with_median=False):
    if with_median:
        median = df[method_name].median()
        df.loc[pd.isna(df[method_name]), method_name] = median  
        # filling with median in the missing prediction score location
    else: 
        df = df[~pd.isna(df[method_name])]  # taking only non-NAN values
    return df
    

def compute_performance_metrics_for_patho(variants_df, method_name, positive_cls, negative_cls, n_runs, home_dir, fill_missing_with_median=False):
    metric_scores = []
    for i in range(n_runs):
        if method_name == "random_classifier":
            variants_df[method_name] = [random.uniform(0, 1) for i in range(variants_df.shape[0])]
            
        sampled_df = variants_df.groupby(by="prot_acc_version").sample(1).copy()
        sampled_df = fill_missing_values(sampled_df, method_name, fill_missing_with_median)            
        sampled_df["pred"] = sampled_df[method_name].copy()
        sampled_df.loc[(sampled_df["class"]==positive_cls), "class_numeric"] = 1
        sampled_df.loc[(sampled_df["class"]==negative_cls), "class_numeric"] = 0 
        
        if method_name in ["phyloP17way_primate", "phastCons17way_primate"]:
            th_max = 0.5
            sampled_df.loc[sampled_df["pred"] >= th_max, "pred"] = 1
            sampled_df.loc[sampled_df["pred"] < th_max, "pred"] = 0

        auc_roc_score, _ = get_auc_roc_score(sampled_df)
        # ks_statistic, ks_pvalue = get_KS_test_score(sampled_df)
        auc_pr_score, precisions, recalls, thresholds = get_auc_pr_score(sampled_df)
        f1_max, th_max, precisions, recalls, thresholds = get_f1max_and_th(
            precisions, recalls, thresholds
        )

        if positive_cls == "Likely-pathogenic":
            th_max = get_pathogenic_cls_analysis_threshold(method_name, home_dir)
        if method_name in ["phyloP17way_primate", "phastCons17way_primate"]:
            th_max = 0.5

        precision = get_precision_score(sampled_df, th_max)
        recall = get_recall_score(sampled_df, th_max)
        accuracy = get_accuracy_score(sampled_df, th_max)
        balanced_accuracy = get_balanced_accuracy_score(sampled_df, th_max)
        mcc = get_matthews_corrcoef(sampled_df, th_max)
        
        metric_scores.append(
            [
                auc_roc_score,
                auc_pr_score,
                f1_max,
                th_max,
                precision,
                recall,
                accuracy,
                balanced_accuracy,
                mcc,
            ]
        )
        # print()
        # if i==0: break
    return metric_scores


variants_df.loc[(result_df["class"]=="Common") | (result_df["class"]=="Rare"), "class"] = negative_cls # putting negative class level at Common and Rare level
performance_scores_dict = {}

for i, method_name in enumerate(all_method_names):
    # method_name = "esm1b_t33_650M_UR50S"
    performance_scores_dict[method_name] = compute_performance_metrics_for_patho(variants_df, method_name, positive_cls, negative_cls, n_runs, home_dir, fill_missing_with_median=True)
    # if i==1:break
write_metrics_outputs(performance_scores_dict, output_file=home_dir+f"data/performance_analysis_patho/{task}_{positive_cls}_vs_{negative_cls}.tsv")